In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

In [11]:
polusi = pd.read_csv('data_cleaned_avg.csv')
polusi.head()

,Address,date,SO2,NO2,O3,CO,PM10,PM2.5
0,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-01,0.005071,0.052786,0.004714,1.042857,78.642857,70.142857
1,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-02,0.005375,0.043000,0.017188,1.006250,101.750000,88.875000
2,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-03,0.005222,0.041556,0.015556,0.866667,72.833333,60.500000
3,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-04,0.005167,0.053722,0.010833,0.944444,48.833333,39.444444
4,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-05,0.003905,0.039571,0.012143,0.680952,29.000000,22.285714


In [12]:
numeric_var = list(polusi.dtypes[polusi.dtypes != "object"].index)
polusi_num = polusi[numeric_var]
polusi_num.head()

,SO2,NO2,O3,CO,PM10,PM2.5
0,0.005071,0.052786,0.004714,1.042857,78.642857,70.142857
1,0.005375,0.043000,0.017188,1.006250,101.750000,88.875000
2,0.005222,0.041556,0.015556,0.866667,72.833333,60.500000
3,0.005167,0.053722,0.010833,0.944444,48.833333,39.444444
4,0.003905,0.039571,0.012143,0.680952,29.000000,22.285714


In [13]:
polusi_num.count()

SO2      26564
NO2      26564
O3       26564
CO       26564
PM10     26564
PM2.5    26564
dtype: int64

In [14]:
limiter = pd.read_csv('Measurement_item_info.csv',index_col=False)
limiter = limiter.drop(['Item code','Unit of measurement'], axis=1)
limiter.head()

,Item name,Good,Normal,Bad,Very bad
0,SO2,0.02,0.05,0.15,1.0
1,NO2,0.03,0.06,0.20,2.0
2,CO,2.00,9.00,15.00,50.0
3,O3,0.03,0.09,0.15,0.5
4,PM10,30.00,80.00,150.00,600.0


In [15]:
limiter_transposed = limiter.set_index('Item name').T
limiter_transposed.head()

Item name,SO2,NO2,CO,O3,PM10,PM2.5
Good,0.02,0.03,2.0,0.03,30.0,15.0
Normal,0.05,0.06,9.0,0.09,80.0,35.0
Bad,0.15,0.20,15.0,0.15,150.0,75.0
Very bad,1.00,2.00,50.0,0.50,600.0,500.0


In [17]:
listnew = []
for i in range(0,26564):
    if limiter_transposed['SO2'][0] > polusi_num['SO2'][i] and limiter_transposed['NO2'][0] > polusi_num['NO2'][i] and limiter_transposed['CO'][0] > polusi_num['CO'][i] and limiter_transposed['O3'][0] > polusi_num['O3'][i] and limiter_transposed['PM10'][0] > polusi_num['PM10'][i] and limiter_transposed['PM2.5'][0] > polusi_num['PM2.5'][i]:
        listnew.append("Good")
    elif limiter_transposed['SO2'][1] > polusi_num['SO2'][i] and limiter_transposed['NO2'][1] > polusi_num['NO2'][i] and limiter_transposed['CO'][1] > polusi_num['CO'][i] and limiter_transposed['O3'][1] > polusi_num['O3'][i] and limiter_transposed['PM10'][1] > polusi_num['PM10'][i] and limiter_transposed['PM2.5'][1] > polusi_num['PM2.5'][i]:
        listnew.append("Normal")
    elif limiter_transposed['SO2'][2] > polusi_num['SO2'][i] and limiter_transposed['NO2'][2] > polusi_num['NO2'][i] and limiter_transposed['CO'][2] > polusi_num['CO'][i] and limiter_transposed['O3'][2] > polusi_num['O3'][i] and limiter_transposed['PM10'][2] > polusi_num['PM10'][i] and limiter_transposed['PM2.5'][2] > polusi_num['PM2.5'][i]:
        listnew.append("Bad")
    else :
        listnew.append("Very Bad")            

In [18]:
polusi['Category'] = listnew
polusi.head()

,Address,date,SO2,NO2,O3,CO,PM10,PM2.5,Category
0,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-01,0.005071,0.052786,0.004714,1.042857,78.642857,70.142857,Bad
1,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-02,0.005375,0.043000,0.017188,1.006250,101.750000,88.875000,Very Bad
2,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-03,0.005222,0.041556,0.015556,0.866667,72.833333,60.500000,Bad
3,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-04,0.005167,0.053722,0.010833,0.944444,48.833333,39.444444,Bad
4,"10, Poeun-ro 6-gil, Mapo-gu, Seoul, Republic o...",2017-01-05,0.003905,0.039571,0.012143,0.680952,29.000000,22.285714,Normal


In [19]:
polusi.to_csv(r'Measurement_summary_v2.csv', index = False, header=True)